**How many COVID cases will there be?**

Factors:
* COVID case growth rates
* Vaccines timeline (getting EUA, starting distribution, achieving sufficient scale)

In [358]:
# Model variables
GROWTH_DERIVATIVE_FACTOR_LOW = 5
GROWTH_DERIVATIVE_FACTOR_HIGH = 30

DAYS_UNTIL_PEAK_GROWTH_LOW = -30
DAYS_UNTIL_PEAK_GROWTH_HIGH = 60

MISSING_INFECTION_RATE_LOW = 5
MISSING_INFECTION_RATE_HIGH = 30

RANDOM_CASE_FLUCTUATION_LOW = -2000
RANDOM_CASE_FLUCTUATION_HIGH = 4000

DAILY_FOURTH_WAVE_CHANCE = 0.6/365
FOURTH_WAVE_LENGTH_DAYS_LOW = 30
FOURTH_WAVE_LENGTH_DAYS_HIGH = 120

million = 1000000

# These are set to roughly curve fit the estimates from
# https://www.metaculus.com/questions/4066/10m-are-administered-an-efficacious-vaccine/
# https://www.metaculus.com/questions/5727/100m-people-vaccinated-against-covid-19/
# https://www.metaculus.com/questions/5767/1bn-people-vaccinated-against-covid-19/
MONTHLY_VACCINATION_DAILY_AMOUNTS = [np.round(10 * million * (1/30) * (1/12), 0),
                                     np.round(50 * million * (1/30) * (1/12), 0),
                                     np.round(100 * million * (1/30) * (1/12), 0),
                                     np.round(150 * million * (1/30) * (1/12), 0),
                                     np.round(200 * million * (1/30) * (1/12), 0),
                                     np.round(250 * million * (1/30) * (1/12), 0),
                                     np.round(300 * million * (1/30) * (1/12), 0),
                                     np.round(400 * million * (1/30) * (1/12), 0),
                                     np.round(700 * million * (1/30) * (1/12), 0),
                                     np.round(900 * million * (1/30) * (1/12), 0),
                                     np.round(1100 * million * (1/30) * (1/12), 0),
                                     np.round(1200 * million * (1/30) * (1/12), 0)]

VACCINE_DELAY_DAYS_LOW = 1
VACCINE_DELAY_DAYS_HIGH = 30

# Simulation parameters
N_SCENARIOS = 1000
SIM_START_DATE = '2020-11-28'
SIM_END_DATE = '2022-01-01'
CURRENT_COVID_CASES = 61.8 * million
CURRENT_COVID_GROWTH = 571428
CURRENT_COVID_DERIVATIVE = 4000
Q4_2020_START_COVID = 26.02 * million

In [359]:
import math
import random

import pandas as pd
import numpy as np

from scipy import stats
from datetime import datetime, timedelta


def print_pct(pct, digits=0):
    pct = pct * 100
    pct = np.round(pct, digits)
    if pct >= 100:
        if digits == 0:
            val = '>99.0%'
        else:
            val = '>99.'
            for d in range(digits - 1):
                val += '9'
            val += '9%'
    elif pct <= 0:
        if digits == 0:
            val = '<1.0%'
        else:
            val = '<0.'
            for d in range(digits - 1):
                val += '0'
            val += '1%'
    else:
        val = '{}%'.format(pct)
    return val


def get_metaculus_range(rangex, minx=None, maxx=None):
    lower25, mean, upper75 = np.percentile(rangex, [25, 50, 75])
    results = {}
    results['lower25'] = lower25
    results['mean'] = mean
    results['upper75'] = upper75
    if minx:
        results['minx'] = print_pct(sum([x < minx for x in rangex]) / len(rangex))
    if maxx:
        results['maxx'] = print_pct(sum([x > maxx for x in rangex]) / len(rangex))
    return results
    

def normal_sample(low, high, interval):
    if (low > high) or (high < low):
        raise ValueError
    if low == high:
        return low
    else:
        mu = (high + low) / 2
        cdf_value = 0.5 + 0.5 * interval
        normed_sigma = stats.norm.ppf(cdf_value)
        sigma = (high - mu) / normed_sigma
        return np.random.normal(mu, sigma)

    
def lognormal_sample(low, high, interval):
    if (low > high) or (high < low):
        raise ValueError
    if low == high:
        return low
    else:
        log_low = np.log(low)
        log_high = np.log(high)
        mu = (log_high + log_low) / 2
        cdf_value = 0.5 + 0.5 * interval
        normed_sigma = stats.norm.ppf(cdf_value)
        sigma = (log_high - mu) / normed_sigma
        return np.random.lognormal(mu, sigma)

In [360]:
days_until_sim_end_date = (datetime.strptime(SIM_END_DATE, '%Y-%m-%d') - datetime.strptime(SIM_START_DATE, '%Y-%m-%d')).days + 1

dec_7_2020_covid_collector = []
q4_2020_covid_collector = []
y2020_covid_collector = []
q1_2021_covid_collector = []
jan_covid_collector = []
feb_covid_collector = []
mar_covid_collector = []
apr_covid_collector = []
may_covid_collector = []
jun_covid_collector = []
jul_covid_collector = []
aug_covid_collector = []
sep_covid_collector = []
oct_covid_collector = []
nov_covid_collector = []
dec_covid_collector = []
jan_vaccinated_collector = []
feb_vaccinated_collector = []
mar_vaccinated_collector = []
apr_vaccinated_collector = []
may_vaccinated_collector = []
jun_vaccinated_collector = []
jul_vaccinated_collector = []
aug_vaccinated_collector = []
sep_vaccinated_collector = []
oct_vaccinated_collector = []
nov_vaccinated_collector = []
dec_vaccinated_collector = []


for n in range(N_SCENARIOS):
    done = False
    fourth_wave = False
    
    covid = CURRENT_COVID_CASES
    covid_growth = CURRENT_COVID_GROWTH
    
    growth_derivative_factor = np.round(normal_sample(GROWTH_DERIVATIVE_FACTOR_LOW, GROWTH_DERIVATIVE_FACTOR_HIGH, 0.8) + 1, 1)
    if growth_derivative_factor < 0.3:
        growth_derivative_factor = 0.3
     
    peak_growth_days = np.round(normal_sample(DAYS_UNTIL_PEAK_GROWTH_LOW, DAYS_UNTIL_PEAK_GROWTH_HIGH, 0.8), 0)
    if peak_growth_days < 0:
        peak_growth_days = 0
    peak_growth_date = (datetime.today() + timedelta(days=peak_growth_days)).date()
    
    fourth_wave_days_length = np.round(lognormal_sample(FOURTH_WAVE_LENGTH_DAYS_LOW, FOURTH_WAVE_LENGTH_DAYS_HIGH, 0.8), 0)
    if fourth_wave_days_length < 1:
        fourth_wave_days_length = 1
        
    missing_infection_rate = np.round(lognormal_sample(MISSING_INFECTION_RATE_LOW, MISSING_INFECTION_RATE_HIGH, 0.8), 1)
    if missing_infection_rate < 1:
        missing_infection_rate = 1
        
    vaccine_delay_days = np.round(lognormal_sample(VACCINE_DELAY_DAYS_LOW, VACCINE_DELAY_DAYS_HIGH, 0.8), 1)
    if vaccine_delay_days < 0:
        vaccine_delay_days = 0
    
    vaccine_rate = 0
    total_vaccinated = 0
    
    verbose = (n <= 4) or (n % 10 == 0)
    really_verbose = (n <= 4)
    
    if verbose:
        print('-')
        print('-')
        print('## SCENARIO {}/{} ##'.format(n + 1, N_SCENARIOS))
        print('Growth derivative factor: {}'.format(growth_derivative_factor))
        print('Peak growth on: {}'.format(peak_growth_date))
        print('Missing infection rate: {}'.format(missing_infection_rate))
        print('Vaccine start date: {}'.format((datetime(2021, 1, 1) + timedelta(days = vaccine_delay_days)).date()))

    for day in range(days_until_sim_end_date):
        if not done:
            date = str((today + timedelta(days = day)).date())
            if really_verbose:
                print('Day {}: {}'.format(day, date))
            
            # Grow vaccines
            days_since_vaccine_start = (today + timedelta(days = day) - datetime(2021, 1, 1)).days - vaccine_delay_days
            if days_since_vaccine_start >= 0:
                i = math.floor(days_since_vaccine_start / 30)
                if i >= len(MONTHLY_VACCINATION_DAILY_AMOUNTS):
                    i = len(MONTHLY_VACCINATION_DAILY_AMOUNTS) - 1
                daily_vaccines = MONTHLY_VACCINATION_DAILY_AMOUNTS[i]
                old_vaccine_rate = vaccine_rate
                vaccine_rate = np.round(np.random.normal(daily_vaccines, daily_vaccines), 0)
                if vaccine_rate < old_vaccine_rate:
                    vaccine_rate = old_vaccine_rate
                total_vaccinated = np.round(total_vaccinated + vaccine_rate, 0)
            
            if really_verbose:
                print('...Total vaccinated now {} (+{})'.format(total_vaccinated, vaccine_rate))
            
            
            # Grow COVID
            if day > peak_growth_days and (not fourth_wave or day > fourth_wave_end):
                covid_growth -= CURRENT_COVID_DERIVATIVE / growth_derivative_factor
            else:
                covid_growth += CURRENT_COVID_DERIVATIVE / growth_derivative_factor
            covid_growth = np.round(covid_growth, 0)
            old_covid_growth = covid_growth
            if old_covid_growth < 0:
                old_covid_growth = 0
            
            covid_growth -= np.round(vaccine_rate * 0.02, 0)
            if covid_growth < 0:
                covid_growth = 0
            
            random_case_fluctuation = np.round(normal_sample(RANDOM_CASE_FLUCTUATION_LOW, RANDOM_CASE_FLUCTUATION_HIGH, 0.8), 0)
            covid += covid_growth + random_case_fluctuation
            
            if really_verbose:
                print('...Total COVID cases now {} (+{} {} -{})'.format(covid, old_covid_growth, "%+d" % random_case_fluctuation, np.round(vaccine_rate * 0.02, 0)))
                
            if not fourth_wave and np.random.random() <= DAILY_FOURTH_WAVE_CHANCE:
                if really_verbose:
                    print('...COVID FOURTH WAVE!!!')
                fourth_wave = True
                fourth_wave_end = day + fourth_wave_days_length
            
            
            # Record
            if date == '2020-12-07':
                dec7_covid = covid
                if verbose:
                    print('...COVID by 7 Dec: {}'.format(dec7_covid))
                    
            if date == '2020-12-31':
                q4_2020_covid = covid - Q4_2020_START_COVID
                if verbose:
                    print('...Q4 2020 COVID: {}'.format(q4_2020_covid))
                y2020_covid = np.round(covid * missing_infection_rate, 0)
                if verbose:
                    print('...Total 2020 COVID: {}'.format(covid))
                    print('...Total 2020 COVID after missing transform: {}'.format(y2020_covid))
                
            if date == '2021-01-30':
                jan_covid = covid - q4_2020_covid - Q4_2020_START_COVID
                jan_vaccinated = total_vaccinated
                
            if date == '2021-02-28':
                feb_covid = covid - jan_covid - q4_2020_covid - Q4_2020_START_COVID
                feb_vaccinated = total_vaccinated
                
            if date == '2021-03-31':
                mar_covid = covid - feb_covid - jan_covid - q4_2020_covid - Q4_2020_START_COVID
                q1_2021_covid = covid - q4_2020_covid - Q4_2020_START_COVID
                mar_vaccinated = total_vaccinated
                if verbose:
                    print('...Q1 2021 COVID: {}'.format(q1_2021_covid))
            
            if date == '2021-04-30':
                apr_vaccinated = total_vaccinated
                apr_covid = covid - mar_covid - feb_covid - jan_covid - q4_2020_covid - Q4_2020_START_COVID
                
            if date == '2021-05-31':
                may_vaccinated = total_vaccinated
                may_covid = covid - apr_covid - mar_covid - feb_covid - jan_covid - q4_2020_covid - Q4_2020_START_COVID
            
            if date == '2021-06-30':
                jun_vaccinated = total_vaccinated
                jun_covid = covid - may_covid - apr_covid - mar_covid - feb_covid - jan_covid - q4_2020_covid - Q4_2020_START_COVID
            
            if date == '2021-07-31':
                jul_vaccinated = total_vaccinated
                jul_covid = covid - jun_covid - may_covid - apr_covid - mar_covid - feb_covid - jan_covid - q4_2020_covid - Q4_2020_START_COVID
                
            if date == '2021-08-31':
                aug_vaccinated = total_vaccinated
                aug_covid = covid - jul_covid - jun_covid - may_covid - apr_covid - mar_covid - feb_covid - jan_covid - q4_2020_covid - Q4_2020_START_COVID
            
            if date == '2021-09-30':
                sep_vaccinated = total_vaccinated
                sep_covid = covid - aug_covid - jul_covid - jun_covid - may_covid - apr_covid - mar_covid - feb_covid - jan_covid - q4_2020_covid - Q4_2020_START_COVID
                
            if date == '2021-10-31':
                oct_vaccinated = total_vaccinated
                oct_covid = covid - sep_covid - aug_covid - jul_covid - jun_covid - may_covid - apr_covid - mar_covid - feb_covid - jan_covid - q4_2020_covid - Q4_2020_START_COVID
            
            if date == '2021-11-30':
                nov_vaccinated = total_vaccinated
                nov_covid = covid - oct_covid - sep_covid - aug_covid - jul_covid - jun_covid - may_covid - apr_covid - mar_covid - feb_covid - jan_covid - q4_2020_covid - Q4_2020_START_COVID
            
            if date == '2021-12-31':
                dec_vaccinated = total_vaccinated
                dec_covid = covid - nov_covid - oct_covid - sep_covid - aug_covid - jul_covid - jun_covid - may_covid - apr_covid - mar_covid - feb_covid - jan_covid - q4_2020_covid - Q4_2020_START_COVID
   
    dec_7_2020_covid_collector.append(dec7_covid)
    q4_2020_covid_collector.append(q4_2020_covid)
    y2020_covid_collector.append(y2020_covid)
    q1_2021_covid_collector.append(q1_2021_covid)
    jan_covid_collector.append(jan_covid)
    feb_covid_collector.append(feb_covid)
    mar_covid_collector.append(mar_covid)
    apr_covid_collector.append(apr_covid)
    may_covid_collector.append(may_covid)
    jun_covid_collector.append(jun_covid)
    jul_covid_collector.append(jul_covid)
    aug_covid_collector.append(aug_covid)
    sep_covid_collector.append(sep_covid)
    oct_covid_collector.append(oct_covid)
    nov_covid_collector.append(nov_covid)
    dec_covid_collector.append(dec_covid)
    jan_vaccinated_collector.append(jan_vaccinated)
    feb_vaccinated_collector.append(feb_vaccinated)
    mar_vaccinated_collector.append(mar_vaccinated)
    apr_vaccinated_collector.append(apr_vaccinated)
    may_vaccinated_collector.append(may_vaccinated)
    jun_vaccinated_collector.append(jun_vaccinated)
    jul_vaccinated_collector.append(jul_vaccinated)
    aug_vaccinated_collector.append(aug_vaccinated)
    sep_vaccinated_collector.append(sep_vaccinated)
    oct_vaccinated_collector.append(oct_vaccinated)
    nov_vaccinated_collector.append(nov_vaccinated)
    dec_vaccinated_collector.append(dec_vaccinated)


-
-
## SCENARIO 1/1000 ##
Growth derivative factor: 17.0
Peak growth on: 2020-12-14
Missing infection rate: 12.1
Vaccine start date: 2021-01-04
Day 0: 2020-12-01
...Total vaccinated now 0 (+0)
...Total COVID cases now 62370018.0 (+571663.0 -1645 -0.0)
Day 1: 2020-12-02
...Total vaccinated now 0 (+0)
...Total COVID cases now 62943897.0 (+571898.0 +1981 -0.0)
Day 2: 2020-12-03
...Total vaccinated now 0 (+0)
...Total COVID cases now 63518712.0 (+572133.0 +2682 -0.0)
Day 3: 2020-12-04
...Total vaccinated now 0 (+0)
...Total COVID cases now 64086521.0 (+572368.0 -4559 -0.0)
Day 4: 2020-12-05
...Total vaccinated now 0 (+0)
...Total COVID cases now 64659109.0 (+572603.0 -15 -0.0)
Day 5: 2020-12-06
...Total vaccinated now 0 (+0)
...Total COVID cases now 65230878.0 (+572838.0 -1069 -0.0)
Day 6: 2020-12-07
...Total vaccinated now 0 (+0)
...Total COVID cases now 65805656.0 (+573073.0 +1705 -0.0)
...COVID by 7 Dec: 65805656.0
Day 7: 2020-12-08
...Total vaccinated now 0 (+0)
...Total COVID cases no

Day 383: 2021-12-19
...Total vaccinated now 1111776897.0 (+9694470.0)
...Total COVID cases now 114659499.0 (+0 +140 -193889.0)
Day 384: 2021-12-20
...Total vaccinated now 1121471367.0 (+9694470.0)
...Total COVID cases now 114663873.0 (+0 +4374 -193889.0)
Day 385: 2021-12-21
...Total vaccinated now 1131165837.0 (+9694470.0)
...Total COVID cases now 114666152.0 (+0 +2279 -193889.0)
Day 386: 2021-12-22
...Total vaccinated now 1140860307.0 (+9694470.0)
...Total COVID cases now 114669147.0 (+0 +2995 -193889.0)
Day 387: 2021-12-23
...Total vaccinated now 1150554777.0 (+9694470.0)
...Total COVID cases now 114668818.0 (+0 -329 -193889.0)
Day 388: 2021-12-24
...Total vaccinated now 1160249247.0 (+9694470.0)
...Total COVID cases now 114672029.0 (+0 +3211 -193889.0)
Day 389: 2021-12-25
...Total vaccinated now 1169943717.0 (+9694470.0)
...Total COVID cases now 114670372.0 (+0 -1657 -193889.0)
Day 390: 2021-12-26
...Total vaccinated now 1179638187.0 (+9694470.0)
...Total COVID cases now 114674824.0

Day 312: 2021-10-09
...Total vaccinated now 438231330.0 (+4806614.0)
...Total COVID cases now 121555056.0 (+0 -490 -96132.0)
Day 313: 2021-10-10
...Total vaccinated now 443037944.0 (+4806614.0)
...Total COVID cases now 121554513.0 (+0 -543 -96132.0)
Day 314: 2021-10-11
...Total vaccinated now 447844558.0 (+4806614.0)
...Total COVID cases now 121559886.0 (+0 +5373 -96132.0)
Day 315: 2021-10-12
...Total vaccinated now 452847611.0 (+5003053.0)
...Total COVID cases now 121556914.0 (+0 -2972 -100061.0)
Day 316: 2021-10-13
...Total vaccinated now 458849718.0 (+6002107.0)
...Total COVID cases now 121554393.0 (+0 -2521 -120042.0)
Day 317: 2021-10-14
...Total vaccinated now 464851825.0 (+6002107.0)
...Total COVID cases now 121559114.0 (+0 +4721 -120042.0)
Day 318: 2021-10-15
...Total vaccinated now 470853932.0 (+6002107.0)
...Total COVID cases now 121559992.0 (+0 +878 -120042.0)
Day 319: 2021-10-16
...Total vaccinated now 476856039.0 (+6002107.0)
...Total COVID cases now 121560288.0 (+0 +296 -1

...Total COVID cases now 114373382.0 (+0 +1547 -49757.0)
Day 241: 2021-07-30
...Total vaccinated now 220019107.0 (+2487832.0)
...Total COVID cases now 114375195.0 (+0 +1813 -49757.0)
Day 242: 2021-07-31
...Total vaccinated now 222506939.0 (+2487832.0)
...Total COVID cases now 114372094.0 (+0 -3101 -49757.0)
Day 243: 2021-08-01
...Total vaccinated now 224994771.0 (+2487832.0)
...Total COVID cases now 114370717.0 (+0 -1377 -49757.0)
Day 244: 2021-08-02
...Total vaccinated now 227482603.0 (+2487832.0)
...Total COVID cases now 114371055.0 (+0 +338 -49757.0)
Day 245: 2021-08-03
...Total vaccinated now 229970435.0 (+2487832.0)
...Total COVID cases now 114372486.0 (+0 +1431 -49757.0)
Day 246: 2021-08-04
...Total vaccinated now 232458267.0 (+2487832.0)
...Total COVID cases now 114371946.0 (+0 -540 -49757.0)
Day 247: 2021-08-05
...Total vaccinated now 234946099.0 (+2487832.0)
...Total COVID cases now 114373640.0 (+0 +1694 -49757.0)
Day 248: 2021-08-06
...Total vaccinated now 237433931.0 (+24878

Day 170: 2021-05-20
...Total vaccinated now 68106468.0 (+1441674.0)
...Total COVID cases now 128414561.0 (+0 +2779 -28833.0)
Day 171: 2021-05-21
...Total vaccinated now 69548142.0 (+1441674.0)
...Total COVID cases now 128416835.0 (+0 +2274 -28833.0)
Day 172: 2021-05-22
...Total vaccinated now 70989816.0 (+1441674.0)
...Total COVID cases now 128417503.0 (+0 +668 -28833.0)
Day 173: 2021-05-23
...Total vaccinated now 72431490.0 (+1441674.0)
...Total COVID cases now 128416922.0 (+0 -581 -28833.0)
Day 174: 2021-05-24
...Total vaccinated now 73873164.0 (+1441674.0)
...Total COVID cases now 128417679.0 (+0 +757 -28833.0)
Day 175: 2021-05-25
...Total vaccinated now 75314838.0 (+1441674.0)
...Total COVID cases now 128418527.0 (+0 +848 -28833.0)
Day 176: 2021-05-26
...Total vaccinated now 76756512.0 (+1441674.0)
...Total COVID cases now 128416669.0 (+0 -1858 -28833.0)
Day 177: 2021-05-27
...Total vaccinated now 78198186.0 (+1441674.0)
...Total COVID cases now 128421114.0 (+0 +4445 -28833.0)
Day 

...Total vaccinated now 17104249.0 (+760576.0)
...Total COVID cases now 112271183.0 (+238589.0 +905 -15212.0)
Day 100: 2021-03-11
...Total vaccinated now 17864825.0 (+760576.0)
...Total COVID cases now 112480756.0 (+223129.0 +1656 -15212.0)
Day 101: 2021-03-12
...Total vaccinated now 18625401.0 (+760576.0)
...Total COVID cases now 112673770.0 (+207669.0 +557 -15212.0)
Day 102: 2021-03-13
...Total vaccinated now 19385977.0 (+760576.0)
...Total COVID cases now 112850769.0 (+192209.0 +2 -15212.0)
Day 103: 2021-03-14
...Total vaccinated now 20146553.0 (+760576.0)
...Total COVID cases now 113015577.0 (+176749.0 +3271 -15212.0)
Day 104: 2021-03-15
...Total vaccinated now 20907129.0 (+760576.0)
...Total COVID cases now 113165599.0 (+161289.0 +3945 -15212.0)
Day 105: 2021-03-16
...Total vaccinated now 21667705.0 (+760576.0)
...Total COVID cases now 113296829.0 (+145829.0 +613 -15212.0)
Day 106: 2021-03-17
...Total vaccinated now 22428281.0 (+760576.0)
...Total COVID cases now 113414411.0 (+130

-
-
## SCENARIO 11/1000 ##
Growth derivative factor: 20.6
Peak growth on: 2020-12-22
Missing infection rate: 18.5
Vaccine start date: 2021-01-10
...COVID by 7 Dec: 65821843.0
...Q4 2020 COVID: 53602523.0
...Total 2020 COVID: 79622523.0
...Total 2020 COVID after missing transform: 1473016676.0
...Q1 2021 COVID: 39156394.0
-
-
## SCENARIO 21/1000 ##
Growth derivative factor: 23.3
Peak growth on: 2021-01-12
Missing infection rate: 29.6
Vaccine start date: 2021-01-08
...COVID by 7 Dec: 65805342.0
...Q4 2020 COVID: 53601149.0
...Total 2020 COVID: 79621149.0
...Total 2020 COVID after missing transform: 2356786010.0
...Q1 2021 COVID: 38009549.0
-
-
## SCENARIO 31/1000 ##
Growth derivative factor: 22.5
Peak growth on: 2021-01-16
Missing infection rate: 6.8
Vaccine start date: 2021-01-02
...COVID by 7 Dec: 65806727.0
...Q4 2020 COVID: 53604110.0
...Total 2020 COVID: 79624110.0
...Total 2020 COVID after missing transform: 541443948.0
...Q1 2021 COVID: 34622871.0
-
-
## SCENARIO 41/1000 ##
Growth

-
-
## SCENARIO 271/1000 ##
Growth derivative factor: 0.3
Peak growth on: 2020-12-01
Missing infection rate: 10.8
Vaccine start date: 2021-01-02
...COVID by 7 Dec: 65621035.0
...Q4 2020 COVID: 47750545.0
...Total 2020 COVID: 73770545.0
...Total 2020 COVID after missing transform: 796721886.0
...Q1 2021 COVID: 1243609.0
-
-
## SCENARIO 281/1000 ##
Growth derivative factor: 6.9
Peak growth on: 2020-12-08
Missing infection rate: 33.1
Vaccine start date: 2021-01-03
...COVID by 7 Dec: 65819567.0
...Q4 2020 COVID: 53460993.0
...Total 2020 COVID: 79480993.0
...Total 2020 COVID after missing transform: 2630820868.0
...Q1 2021 COVID: 32839083.0
-
-
## SCENARIO 291/1000 ##
Growth derivative factor: 16.4
Peak growth on: 2021-01-20
Missing infection rate: 19.8
Vaccine start date: 2021-01-06
...COVID by 7 Dec: 65817868.0
...Q4 2020 COVID: 53644717.0
...Total 2020 COVID: 79664717.0
...Total 2020 COVID after missing transform: 1577361397.0
...Q1 2021 COVID: 37668196.0
-
-
## SCENARIO 301/1000 ##
Grow

-
-
## SCENARIO 531/1000 ##
Growth derivative factor: 32.3
Peak growth on: 2020-12-01
Missing infection rate: 10.9
Vaccine start date: 2021-01-06
...COVID by 7 Dec: 65812717.0
...Q4 2020 COVID: 53495301.0
...Total 2020 COVID: 79515301.0
...Total 2020 COVID after missing transform: 866716781.0
...Q1 2021 COVID: 34360062.0
-
-
## SCENARIO 541/1000 ##
Growth derivative factor: 21.0
Peak growth on: 2021-02-13
Missing infection rate: 7.1
Vaccine start date: 2021-01-14
...COVID by 7 Dec: 65811439.0
...Q4 2020 COVID: 53613854.0
...Total 2020 COVID: 79633854.0
...Total 2020 COVID after missing transform: 565400363.0
...Q1 2021 COVID: 41920834.0
-
-
## SCENARIO 551/1000 ##
Growth derivative factor: 21.0
Peak growth on: 2020-12-01
Missing infection rate: 12.5
Vaccine start date: 2021-01-13
...COVID by 7 Dec: 65804574.0
...Q4 2020 COVID: 53434121.0
...Total 2020 COVID: 79454121.0
...Total 2020 COVID after missing transform: 993176512.0
...Q1 2021 COVID: 37803049.0
-
-
## SCENARIO 561/1000 ##
Grow

-
-
## SCENARIO 791/1000 ##
Growth derivative factor: 26.4
Peak growth on: 2020-12-01
Missing infection rate: 12.5
Vaccine start date: 2021-01-01
...COVID by 7 Dec: 65806381.0
...Q4 2020 COVID: 53457318.0
...Total 2020 COVID: 79477318.0
...Total 2020 COVID after missing transform: 993466475.0
...Q1 2021 COVID: 32021810.0
-
-
## SCENARIO 801/1000 ##
Growth derivative factor: 9.4
Peak growth on: 2020-12-24
Missing infection rate: 6.1
Vaccine start date: 2021-01-03
...COVID by 7 Dec: 65823411.0
...Q4 2020 COVID: 53719631.0
...Total 2020 COVID: 79739631.0
...Total 2020 COVID after missing transform: 486411749.0
...Q1 2021 COVID: 34252458.0
-
-
## SCENARIO 811/1000 ##
Growth derivative factor: 24.2
Peak growth on: 2020-12-25
Missing infection rate: 38.6
Vaccine start date: 2021-04-04
...COVID by 7 Dec: 65811656.0
...Q4 2020 COVID: 53587369.0
...Total 2020 COVID: 79607369.0
...Total 2020 COVID after missing transform: 3072844443.0
...Q1 2021 COVID: 51137710.0
-
-
## SCENARIO 821/1000 ##
Grow

## Total Worldwide COVID by 7 Dec

For calibration

In [361]:
get_metaculus_range(dec_7_2020_covid_collector)

{'lower25': 65805029.5, 'mean': 65810523.0, 'upper75': 65817022.75}

## 2020 Total COVID Cases with Missing Case Estimation

For https://www.metaculus.com/questions/3892/how-many-infections-of-sars-cov-2-novel-coronavirus-will-be-estimated-to-have-occurred-worldwide-before-2021/

In [362]:
get_metaculus_range(y2020_covid_collector, minx=1000000)

{'lower25': 605114203.75,
 'mean': 953686470.0,
 'upper75': 1481070956.25,
 'minx': '<1.0%'}

## 2020 Q4 Recorded COVID Cases

For https://www.metaculus.com/questions/3767/how-many-new-cases-of-covid-19-in-the-4th-quarter-of-2020/

In [363]:
get_metaculus_range(q4_2020_covid_collector, minx=1000, maxx=1000000000)

{'lower25': 53454873.75,
 'mean': 53579778.0,
 'upper75': 53634635.25,
 'minx': '<1.0%',
 'maxx': '<1.0%'}

## 2021 Q1 Recorded COVID Cases

For https://www.metaculus.com/questions/3768/how-many-new-cases-of-covid-19-in-the-1st-quarter-of-2021/

In [364]:
get_metaculus_range(q1_2021_covid_collector, minx=1000, maxx=1000000000)

{'lower25': 33968372.0,
 'mean': 36532861.5,
 'upper75': 40601495.5,
 'minx': '<1.0%',
 'maxx': '<1.0%'}

## Vaccinations

In [365]:
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

collectors = [jan_vaccinated_collector, feb_vaccinated_collector, mar_vaccinated_collector, apr_vaccinated_collector,
              may_vaccinated_collector, jun_vaccinated_collector, jul_vaccinated_collector, aug_vaccinated_collector,
              sep_vaccinated_collector, oct_vaccinated_collector, nov_vaccinated_collector, dec_vaccinated_collector]

for i, month in enumerate(months):
    print('total vaccinated by EOM {} 2021: {}'.format(month, np.round(np.array(collectors[i]).mean()), 0))

total vaccinated by EOM Jan 2021: 1382055.0
total vaccinated by EOM Feb 2021: 8595277.0
total vaccinated by EOM Mar 2021: 26464935.0
total vaccinated by EOM Apr 2021: 55391097.0
total vaccinated by EOM May 2021: 98121031.0
total vaccinated by EOM Jun 2021: 152563317.0
total vaccinated by EOM Jul 2021: 222226645.0
total vaccinated by EOM Aug 2021: 311982276.0
total vaccinated by EOM Sep 2021: 449973601.0
total vaccinated by EOM Oct 2021: 652897050.0
total vaccinated by EOM Nov 2021: 902851070.0
total vaccinated by EOM Dec 2021: 1205810425.0


In [366]:
# https://www.metaculus.com/questions/4066/10m-are-administered-an-efficacious-vaccine/
for i, month in enumerate(months):
    print('10M vaccinated by EOM {} 2021: {}'.format(month, print_pct(np.sum([x > 10 * million for x in collectors[i]]) / len(collectors[i]))))
    
print('-')
for i, month in enumerate(months):
    print('100M vaccinated by EOM {} 2021: {}'.format(month, print_pct(np.sum([x > 100 * million for x in collectors[i]]) / len(collectors[i]))))

print('-')
for i, month in enumerate(months):
    print('1B vaccinated by EOM {} 2021: {}'.format(month, print_pct(np.sum([x > 1000 * million for x in collectors[i]]) / len(collectors[i]))))

10M vaccinated by EOM Jan 2021: <1.0%
10M vaccinated by EOM Feb 2021: 42.0%
10M vaccinated by EOM Mar 2021: 93.0%
10M vaccinated by EOM Apr 2021: 97.0%
10M vaccinated by EOM May 2021: 99.0%
10M vaccinated by EOM Jun 2021: 99.0%
10M vaccinated by EOM Jul 2021: >99.0%
10M vaccinated by EOM Aug 2021: >99.0%
10M vaccinated by EOM Sep 2021: >99.0%
10M vaccinated by EOM Oct 2021: >99.0%
10M vaccinated by EOM Nov 2021: >99.0%
10M vaccinated by EOM Dec 2021: >99.0%
-
100M vaccinated by EOM Jan 2021: <1.0%
100M vaccinated by EOM Feb 2021: <1.0%
100M vaccinated by EOM Mar 2021: <1.0%
100M vaccinated by EOM Apr 2021: <1.0%
100M vaccinated by EOM May 2021: 59.0%
100M vaccinated by EOM Jun 2021: 94.0%
100M vaccinated by EOM Jul 2021: 97.0%
100M vaccinated by EOM Aug 2021: 98.0%
100M vaccinated by EOM Sep 2021: 99.0%
100M vaccinated by EOM Oct 2021: >99.0%
100M vaccinated by EOM Nov 2021: >99.0%
100M vaccinated by EOM Dec 2021: >99.0%
-
1B vaccinated by EOM Jan 2021: <1.0%
1B vaccinated by EOM Feb 2

## 2021 Peak Month

For https://www.metaculus.com/questions/5432/month-with-peak-of-covid-19-cases-in-2021/

In [367]:
max_month_collector = []

for i in range(N_SCENARIOS):
    covid_months = [jan_covid_collector[i], feb_covid_collector[i], mar_covid_collector[i], apr_covid_collector[i],
                    may_covid_collector[i], jun_covid_collector[i], jul_covid_collector[i], aug_covid_collector[i],
                    sep_covid_collector[i], oct_covid_collector[i], nov_covid_collector[i], dec_covid_collector[i]]
    max_month = months[covid_months.index(max(covid_months))]
    max_month_collector.append(max_month)

pd.Series(max_month_collector).value_counts(normalize=True) * 100

Jan    96.7
Mar     2.7
Feb     0.4
May     0.1
Aug     0.1
dtype: float64